<img width="8%" alt="Naas.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Naas.png" style="border-radius: 15%">

# Naas - Send notification

**Tags:** #naas #emailbuilder #snippet #operations

**Author:** [Florent Ravenel](https://www.linkedin.com/in/ACoAABCNSioBW3YZHc2lBHVG0E_TXYWitQkmwog/)

**Description:** This notebook send a notification with your latest posts performance.

## Input

### Import libraries

In [ ]:
from naas_drivers import emailbuilder, gsheet
import naas
import os
import naas_data_product
from datetime import date, datetime, timedelta
import pandas as pd

### Setup variables
**Inputs**
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `input_dir`: Input directory to retrieve file from.
- `datalake_dir`: Datalake directory to retrieve plugin json file.

**Outputs**
- `output_dir`: This variable holds a relative path to the directory where output files will be stored. 
- `email_to`: This variable is used for storing a list of email addresses that will receive the notification email.

In [ ]:
# Inputs
input_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "sales-engine", date.today().isoformat())
spreadsheet_url = naas.secret.get("ABI_SPREADSHEET") or "YOUR_GOOGLE_SPREADSHEET_URL"
sheet_name_input = "DEALS"
datalake_dir = naas_data_product.OUTPUTS_PATH
input_file = "deals"
input_image = "deals_trend.png"
input_html = "deals_trend.html"

# Outputs
output_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "sales-engine", date.today().isoformat())
email_to = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entity"), "emails") or ""

## Model

### Set outputs

In [ ]:
entity_name = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entity"), "entity_name") or ""
timestamp_email = datetime.now(TIMEZONE).strftime('%Y-%m-%d %H:%M:%S')
timestamp_csv = datetime.now(TIMEZONE).strftime('%Y%m%d%H%M%S')
email_subject = f"⚡️ Sales Assistant Report - {entity_name} ({timestamp_email})"
csv_output = os.path.join(output_dir, f"{timestamp_csv}_{TW}_sales-assistant-report_{entity_name.lower().replace(' ', '_')}.csv")
print("- CSV output:", csv_output)

### Get assets

In [ ]:
image_url = get_image_asset(input_dir, input_image)
print("Image URL:", image_url)

image_html = get_image_asset(input_dir, input_html)
print("Image HTML:", image_html)

plugin_url = naas.asset.find(os.path.join(datalake_dir, "plugins", "⚡️_sales_assitant.json"))
print("Plugin URL:", plugin_url)

### Get deals

In [ ]:
df_input = pload(input_dir, input_file)    
if df_input is None:
    df_input = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name_input)

# Filter on this week and last week
df_input = df_input[df_input["SCENARIO"].isin([TW, LW])]

# Display result
df_input.to_csv(csv_output, index=False)
print("Rows:", len(df_input))
df_input.head(1)

### Get today's new deals

In [ ]:
df_today = df_input.copy()
df_today["DEAL_CREATED_DATE"] = pd.to_datetime(df_today["DEAL_CREATED_DATE"].str[:19])
df_today = df_today[df_today["DEAL_CREATED_DATE"] == date.today().isoformat()]
print("New deals:", len(df_today))

### Prep companies for email

In [ ]:
df_companies = df_today.copy()

staff_maping = {
    "Solopreneur": 1,
    "Micro Team": 2,
    "Small Company": 3,
    "Medium Company": 4,
    "Large Company": 5,
    "Enterprise Level": 6,
    "Major Corporation": 7,
    "Global Corporation": 8,
    "Mega Corporation": 9,
    "NA": 0,
}

to_group = [
    "ORG_NAME",
    "ORG_LINKEDIN_URL",
    "ORG_STAFF_RANGE_NAME",
    "ORG_COUNTRY",
]
to_agg = {
    "DEAL_SCORE": "sum"
}
df_companies = df_companies.groupby(to_group, as_index=False).agg(to_agg)
df_companies = df_companies[(df_companies["ORG_NAME"] != "TBD") & (df_companies["ORG_STAFF_RANGE_NAME"] != "NA")]
df_companies["ORG_SIZE_ORDER"] = df_companies["ORG_STAFF_RANGE_NAME"].map(staff_maping)
df_companies = df_companies.sort_values(by=["ORG_SIZE_ORDER"], ascending=[False]).reset_index(drop=True)

companies = []
for x in df_companies["ORG_STAFF_RANGE_NAME"].unique():
    company = f"{x}:"
    tmp_df = df_companies[df_companies["ORG_STAFF_RANGE_NAME"] == x].reset_index(drop=True)
    for row in tmp_df.itertuples():
        index = row.Index
        name = row.ORG_NAME
        url = row.ORG_LINKEDIN_URL
        company = f"{company}<ul><a href='{url}'>{name}</a></ul>"
    companies.append(company)
if len(companies) == 0:
    companies = [""]
print(companies)

### Prep messagings option for email

In [ ]:
df_messagings = df_today.copy()

messagings = []
for row in df_messagings.itertuples():
    index = row.Index
    fullname = row.PEOPLE_FULLNAME
    profile_url = row.PEOPLE_PROFILE_URL
    occupation = row.PEOPLE_OCCUPATION
    company = row.ORG_NAME
    messagings_options = row.MESSAGING_OPTIONS
    if messagings_options != "TBD":
        option1 = messagings_options.split('1.')[1].split('2.')[0]
        option2 = messagings_options.split('2.')[1].split('3.')[0]
        option3 = messagings_options.split('3.')[1]
        messagings.append(f"<a href='{profile_url}'>{fullname}</a> - '{occupation}' at {company}:<br><span style='font-size: 14px;'><ul>1.{option1}</ul><ul>2.{option2}</ul><ul>3.{option3}</ul></span>") 
    else:
        messagings.append(f"<a href='{profile_url}'>{fullname}</a> - '{occupation}' at {company}")
if len(messagings) == 0:
    messagings = [""]
print(messagings)

### Create Email brief

In [ ]:
def email_brief(
    companies,
    messagings,
    plugin_url,
    image_url,
    image_html,
):
    content = {
        'heading': "⚡️ Sales Assistant",
        "txt_intro": (
            f"Hi {entity_name.split(' ')[0]},<br><br>" "I hope this email finds you well. Here is the latest update on your sales performance."
        ),
        "title_1": emailbuilder.text(
            "Overview", font_size="25px", text_align="center", bold=True
        ),
        "image_1": emailbuilder.image(image_url, image_html),
        "title_2": emailbuilder.text(
            "Performance", font_size="25px", text_align="center", bold=True
        ),
        "text_2": emailbuilder.text(
            "Today's new deals:",
            underline=True,
        ),
        "list_2": emailbuilder.list(messagings),
        "text_1": emailbuilder.text(
            "Companies breadown:",
            underline=True,
        ),
        "list_1": emailbuilder.list(companies),
        "button_1": emailbuilder.button(
            link=(f"https://naas.ai/chat/use?plugin_url={plugin_url}"),
            text="Start Chatting",
            background_color="#181a1c",
        ),
        "footer_cs": emailbuilder.footer_company(naas=True),
    }

    email_content = emailbuilder.generate(display="iframe", **content)
    return email_content


email_content = email_brief(
    companies,
    messagings,
    plugin_url,
    image_url,
    image_html
)

## Output

### Send the email

In [ ]:
if len(email_to) > 0:
    naas.notification.send(
        email_to=email_to,
        subject=email_subject,
        html=email_content,
        files=[csv_output],
    )